In [96]:
import requests
import pandas as pd
import math
import time

In [112]:
api_token = '2gaZEtABi5pxX1tFQeRgudXuWheouJKgYemjwUO58EH15WEKQJGw4NRXeDpbyH+r91fmkmv2ycO9uRH16yBIsQ=='    
user_id = 'cnzHhxx5yqwWiUl'

#Dataframe for saving job IDs
jobs_df = pd.DataFrame(columns = ["ID", "Title", "Company", "Location"])

In [113]:
#CareerOneStop/NLX
#First step is to gather job list; each job IDs will then be queried for specific job details/descriptions in the second step
#The maximum number of results is 500; script must iterate through pages to aggregate all the job IDs.

query_term = 'cybersecurity'
days = '0' #All days
start_page = 1
end_page = 500
url = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{query_term}/United%20States/25/0/0/{start_page}/{end_page}/{days}?showFilters=false'
headers = {'Authorization': 'Bearer ' + api_token,
 'Content-Type': 'application/json'}
results = requests.get(url, headers=headers).json()



In [114]:
#Calculate number of API calls to get all the results
jobs_total = int(results['Jobcount'])
total_pages = math.ceil(int(results['Jobcount'])/500)
print('Number of total jobs: ' + str(jobs_total) , '\nNumber of iterations/pages: ' + str(total_pages))

Number of total jobs: 51261 
Number of iterations/pages: 103


In [118]:
#Loop through all pages of jobs
#Adjust query term and new results will be added to dataframe

query_term = 'cybersecurity'
for i in range(total_pages):

    url = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{query_term}/United%20States/25/0/0/{start_page}/{end_page}/{days}?showFilters=false'
    #Extract job IDs and append to master list
    results = requests.get(url, headers=headers).json()
    for i, value in enumerate(results['Jobs']):
        jobs_df = jobs_df.append(pd.DataFrame([[results['Jobs'][i]['JvId'],results['Jobs'][i]['JobTitle'],results['Jobs'][i]['Company'],results['Jobs'][i]['Location']]], columns = ["ID", "Title", "Company", "Location"] ))
    start_page += 500
    end_page += 500
    time.sleep(1.0)
        

In [120]:
#Simple filter on job titles
cyber_jobs = jobs_df[jobs_df['Title'].str.contains("cyber",case = False)].reset_index(drop = True)

In [127]:
cyber_jobs

,ID,Title,Company,Location
0,7C2321CECB83415B9D1D9854F8867997206,Cybersecurity,US Navy,"Colorado Springs, Colorado"
1,89A5F5B490BA43EF816D7192BB369A1A206,Cybersecurity,Idaho Department of Labor,"Nampa, Idaho"
2,F4B11A16CE5740A898C929513BA7AFE5206,Cybersecurity,US Navy,"Detroit, Michigan"
3,6640417C13B24BBDBEDA468648E26A8C206,Cybersecurity,Arizona Department of Economic Security,"Phoenix, Arizona"
4,EB437BF5A6164CFDAA3A7E1755707369206,Cybersecurity Analyst,Electrosoft,"Washington, District Of Columbia"
...,...,...,...,...
12198,69968E703B1C4FC98D68232D448AEA02206,"Senior Technical Security Analyst, Cyber Risk ...",Splunk,"San Jose, California"
12199,7EAFF35B3E3241B2841D7F412ED6E0A1206,Cyber Security Analyst,ISC Consulting Group,"Augusta, Georgia"
12200,4727F27733084669AE4F359859D13347206,Lead Engineer – Cyberphysical Cloud Operations,National Renewable Energy Laboratory,"Golden, Colorado"
12201,FBA276E4970F4CAE8E81A415A8E5B331206,Senior Cyber Security Analyst - Standardizatio...,UIC Government Services and the Bowhead Family...,"San Antonio, Texas"


In [128]:
#Retrieve job details using list of job IDs
###
#This will generate large number of calls to API!
###
jobs = pd.DataFrame()
for i, value in enumerate(cyber_jobs['ID']):
    job_id = cyber_jobs.iloc[i]['ID']
    try:
        url_getjob = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{job_id}?isHtml=true&enableMetaData=true'
        results = requests.get(url_getjob, headers=headers)
        if results.status_code == 200:
            results = requests.get(url_getjob, headers=headers).json()
            pd.DataFrame([[cyber_jobs.iloc[i]['ID'],results['JobTitle'], results['Company'], results['Description']]]).to_csv('Jobs2.csv', mode='a', index=False, header=False)
            time.sleep(1.5) #throttle the calls down to once per second!
        elif results[0]['Description'] == 'Job No Longer Available':
            continue
    except Exception as inst:
        print(inst)
        #time.sleep(60)
        continue

'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not subscriptable
'Response' object is not 

In [88]:
job_id

'2223A7D528D94C49B6F7D77D55A68FC8206'

In [92]:
cyber_jobs[cyber_jobs['ID'].str.contains("2223A7D528D94C49B6F7D77D55A68FC8206",case = False)]

,ID,Title,Company
598,2223A7D528D94C49B6F7D77D55A68FC8206,Principal-Cybersecurity,AT&T


In [122]:
job_id = '6C41BDC1A0B443618CCBF8E8C75E5472206'
url_getjob = f'https://api.careeronestop.org/v1/jobsearch/{user_id}/{job_id}?isHtml=true&enableMetaData=true'
results = requests.get(url_getjob, headers=headers).json()

In [123]:
results

{'JobTitle': 'Business Analyst III (Cybersecurity)',
 'URL': 'https://de.jobsyn.org/6c41bdc1a0b443618ccbf8e8c75e5472',
 'Company': 'Chick-fil-A, Inc.',
 'Location': 'Atlanta, Georgia',
 'DatePosted': '2020-3-14 8:10 AM',
 'Description': "<p>As a Business Analyst III for Vulnerability Management you will responsible for the Vulnerability Management Program, which includes owning the technology, coordinating scans, validating and prioritizing results, and coordinating the remediations of findings.  As part of your role on the Cybersecurity Consulting team, you will interface with many teams within Information Technology and Chick-fil-A to advise with a cybersecurity perspective and influence cybersecurity with a business perspective.  You will be the bridge between the Cybersecurity team and the rest of the Business.</p>\n\n<ul>\n<li><p>Own Vulnerability Management Program</p></li>\n\n<li><p>Own vulnerability management tools</p></li>\n\n<li><p>Coordinate and run vulnerability scans</p><

In [110]:
results[0]['Description']

'Job No Longer Available'

In [95]:
cyber_jobs[598:]

,ID,Title,Company
598,2223A7D528D94C49B6F7D77D55A68FC8206,Principal-Cybersecurity,AT&T
599,35498B03BC574FCBA61B3BE7831E956B206,Principal-Cybersecurity,AT&T
600,E92068C8442046728B5D79AF2252830B206,Principal-Cybersecurity,AT&T
601,41C72A98A19049878E88862CCDA0724B206,Cybersecurity Engineer,Axient
602,6C41BDC1A0B443618CCBF8E8C75E5472206,Business Analyst III (Cybersecurity),"Chick-fil-A, Inc."
...,...,...,...
12226,69968E703B1C4FC98D68232D448AEA02206,"Senior Technical Security Analyst, Cyber Risk ...",Splunk
12227,7EAFF35B3E3241B2841D7F412ED6E0A1206,Cyber Security Analyst,ISC Consulting Group
12228,4727F27733084669AE4F359859D13347206,Lead Engineer – Cyberphysical Cloud Operations,National Renewable Energy Laboratory
12229,FA4D7FF264C1461D82AAE7036AB316F7206,Senior Cyber Security Analyst - Standardizatio...,Ukpeagvik I&#241;upiat Corporation/Bowhead Fam...


In [99]:
results

{'Jobcount': '51248',
 'Jobs': [{'JvId': '7C2321CECB83415B9D1D9854F8867997206',
   'JobTitle': 'Cybersecurity',
   'Company': 'US Navy',
   'AccquisitionDate': '2022-1-28 9:43 AM',
   'URL': 'https://de.jobsyn.org/7C2321CECB83415B9D1D9854F8867997206',
   'Location': 'Colorado Springs, Colorado',
   'Fc': 'False'},
  {'JvId': '89A5F5B490BA43EF816D7192BB369A1A206',
   'JobTitle': 'Cybersecurity',
   'Company': 'Idaho Department of Labor',
   'AccquisitionDate': '2022-1-29 7:05 AM',
   'URL': 'https://de.jobsyn.org/89A5F5B490BA43EF816D7192BB369A1A206',
   'Location': 'Nampa, Idaho',
   'Fc': 'False'},
  {'JvId': 'F4B11A16CE5740A898C929513BA7AFE5206',
   'JobTitle': 'Cybersecurity',
   'Company': 'US Navy',
   'AccquisitionDate': '2021-9-22 12:08 PM',
   'URL': 'https://de.jobsyn.org/F4B11A16CE5740A898C929513BA7AFE5206',
   'Location': 'Detroit, Michigan',
   'Fc': 'False'},
  {'JvId': '6640417C13B24BBDBEDA468648E26A8C206',
   'JobTitle': 'Cybersecurity',
   'Company': 'Arizona Department 

In [119]:
jobs_df

,ID,Title,Company,Location
0,7C2321CECB83415B9D1D9854F8867997206,Cybersecurity,US Navy,"Colorado Springs, Colorado"
0,89A5F5B490BA43EF816D7192BB369A1A206,Cybersecurity,Idaho Department of Labor,"Nampa, Idaho"
0,F4B11A16CE5740A898C929513BA7AFE5206,Cybersecurity,US Navy,"Detroit, Michigan"
0,6640417C13B24BBDBEDA468648E26A8C206,Cybersecurity,Arizona Department of Economic Security,"Phoenix, Arizona"
0,EB437BF5A6164CFDAA3A7E1755707369206,Cybersecurity Analyst,Electrosoft,"Washington, District Of Columbia"
...,...,...,...,...
0,A6E65B613CB141EE88409C282007F721206,Unit Program Coordinator,Pacific Air Forces,"Eielson AFB, Alaska"
0,0FA430BA7DBD4872A1AC5D2BA1C730F5206,UNIT PROGRAM COORDINATOR,Air Education and Training Command,"Holloman AFB, New Mexico"
0,C8146670A46A4294B67CE7D840712027206,MANAGER,Space and Naval Warfare Systems Command,Point Loma Complex
0,07CDDCF2EFD04BDF813898ACD8F096E3206,IT SPECIALIST (SYSANALYSIS),Space and Naval Warfare Systems Command,"Suffolk, Virginia"
